In [68]:
import pandas as pd
import json

In [69]:
schools = pd.read_json('https://stem-mentoring.firebaseio.com/school.json')
cities = pd.read_json('https://stem-mentoring.firebaseio.com/cities.json')
students = pd.read_json('https://stem-mentoring.firebaseio.com/students.json').T
confirm = pd.read_json('https://stem-mentoring.firebaseio.com/confirm.json').T

{'city': 151, 'id': 1810, 'label': '천안청수고등학교', 'value': '천안청수고등학교'}

In [70]:
students['school_id'] = students.school.map(lambda x:x['id'])
students['school_name'] = students.school.map(lambda x:x['label'])
students['city_id'] = students.school.map(lambda x:x['city'])

In [73]:
students[0:1]

,email,grade,guardian_name,guardian_phone,interests,motivation,name,order,phone,school,timestamp,school_id,school_name,city_id
-KKfhVqZih98PwOf1XYL,taklove0602@gmail.com,2,김용길,01074544131,ERE|CBE,서울대학교 멘토링을 꼭 받아보고 싶습니다. 멘토들과 만남을 하여 제 공부 방법ㅇ 옳...,탁은정,1,01054295229,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T02:49:00.589Z,1810,천안청수고등학교,151


In [83]:
pd.merge(students,cities)[:,['school_name', 'order', 'name', 'grade', 'phone', ]]

,email,grade,guardian_name,guardian_phone,interests,motivation,name,order,phone,school,timestamp,school_id,school_name,city_id,city,province
0,taklove0602@gmail.com,2,김용길,01074544131,ERE|CBE,서울대학교 멘토링을 꼭 받아보고 싶습니다. 멘토들과 만남을 하여 제 공부 방법ㅇ 옳...,탁은정,1,01054295229,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T02:49:00.589Z,1810,천안청수고등학교,151,천안시,충청남도
1,thdms1129@naver.com,2,김용길,01074544131,ERE|CSE|CBE,평소에 공대에 관심이 많습니다.,김소은,2,01076313035,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T02:50:29.721Z,1810,천안청수고등학교,151,천안시,충청남도
2,lemon3009@naver.com,2,김용길,01074544131,ERE|CBE,서울대학교 공과대학에 대하여 더 알아보고 싶기 때문입니다.,이한빛,3,01058823009,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T02:52:30.005Z,1810,천안청수고등학교,151,천안시,충청남도
3,dmsqls0607@naver.com,2,김용길,01074544131,ERE|NE|CBE,평소 공학에 관해 관심이 많았기 떄문입니다,유은빈,4,01043110491,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T02:54:18.048Z,1810,천안청수고등학교,151,천안시,충청남도
4,misswns7@lcloud.com,2,김용길,01074544131,MAE|NAOE|CSE,공학에 관심이 많기 때문입니다.,황준혁,5,01075764606,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T02:56:34.968Z,1810,천안청수고등학교,151,천안시,충청남도
5,cgs06388@naver.com,2,김용길,01074544131,ERE,관심이 이기 때문에,송형근,6,01041557046,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T02:58:41.312Z,1810,천안청수고등학교,151,천안시,충청남도
6,gom3412@naver.com,2,김용길,01074544131,ERE|CBE,공학에 관심이 많아서,최은서,7,01094853087,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T03:00:28.439Z,1810,천안청수고등학교,151,천안시,충청남도
7,kookbeum207@naver.com,2,김용길,01074544131,ERE|CBE,공대에 관심이있어서 지원하게됬습니다.,국범석,8,01034472340,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T03:01:57.850Z,1810,천안청수고등학교,151,천안시,충청남도
8,rabbit0930@daum.net,2,김용길,01074544131,IE,공대에 관심이 있어서요,정성훈,9,01098080270,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T03:04:28.644Z,1810,천안청수고등학교,151,천안시,충청남도
9,zzz1700@naver.com,2,김용길,01074544131,CE|AR,공대에 관심이 있어서,유승희,10,01065501700,"{'city': 151, 'label': '천안청수고등학교', 'value': '천...",2016-06-20T03:33:41.029Z,1810,천안청수고등학교,151,천안시,충청남도


In [89]:
pd.merge(students,cities)[['school_name', 'order', 'name', 'grade', 'phone']].sort_values(['school_name', 'order'])

,school_name,order,name,grade,phone
124,가락고등학교,1,강주연,1,01073049973
125,가락고등학교,2,조승혜,1,01023927003
31,광신고등학교,1,정승원,1,01077672729
32,광신고등학교,2,강우종,1,01032334546
33,광신고등학교,3,강동연,1,01090612268
34,광신고등학교,4,장민우,1,01077102170
35,광신고등학교,5,강성민,1,01056363718
36,광신고등학교,6,이정영,1,01064503284
37,광신고등학교,7,김원종,1,01094277848
38,광신고등학교,8,이준희,1,01055514364


In [95]:
students = pd.read_json('https://stem-mentoring.firebaseio.com/students.json').T
students['school_name'] = students.school.map(lambda x:x['label'])
register_list = students[['school_name', 'order', 'name', 'grade', 'phone']].sort_values(['school_name', 'order'])
register_list.to_excel('명단.xlsx', header=['학교', '순번', '이름', '학년', '연락처'], index=False)